In [176]:
import sys
sys.path.append('../')


In [253]:
df_stats = pd.read_csv('../data_stats/data_stats.csv', index_col = [0])
df_stats_90 = pd.read_csv('../data_stats/data_stats_90_percent.csv', index_col = [0])
df_stats_10 = pd.read_csv('../data_stats/data_stats_10_percent.csv', index_col = [0])
df_stats_science = pd.read_csv('../data_stats/data_stats_science.csv', index_col = [0])

In [264]:
from data_model import Individual
import pandas as pd
import typing as t
from utils import round_nearest
from sys_utils import load_model
from data_model import Individual
from data_model_region import Region
from functions_environment import get_population, get_maddison_data
from bunka_logger import logger
import numpy as np
from tqdm import tqdm
from sklearn import preprocessing

data_path_environment = '../data_env'
path_to_visualisation = '../data_visualisation'
path_to_stats = '../data_stats'
checkpoint_path = '../checkpoints_dev'

 
def get_individual_score(individuals:t.List[Individual]):

    individuals_filtered = [x for x in individuals if x.regions != None]

    df_individuals = [
        {
            "wikidata_id": x.id.wikidata_id,
            "name": x.id.name,
            "year": x.id.birthyear,
            "cultural_score": x.cultural_score,
            "region_code": x.regions,
        }
        for x in individuals_filtered
    ]

    df_individuals = pd.DataFrame(df_individuals)
    df_individuals = df_individuals.dropna()
    df_individuals = df_individuals.explode("region_code").reset_index(drop=True)
    df_individuals['year'] = df_individuals['year'].apply(lambda x : round_nearest(x,10))

    return df_individuals


def create_df_trend(individuals: t.List[Individual], top_percent = None):

    indiv = [x for x in individuals if x.regions != None]
    indiv = [x for x in indiv if x.cultural_score != None]

    df = [
        {
            "wiki_id": x.id.wikidata_id,
            "name": x.id.name,
            "birthyear": x.id.birthyear,
            "region_code": x.regions,
            "range_impact": x.impact_years,
            "cultural_score": x.cultural_score,
        }
        for x in indiv
    ]

    df = pd.DataFrame(df)
    df = df.explode("region_code")
    
    if top_percent is not None:
        groups = df.groupby('region_code')
        sorted_groups = [g.sort_values('cultural_score', ascending=False).reset_index(drop=True) for _, g in groups]
        filtered_groups = [g.iloc[:int(len(g)*(0.1))] for g in sorted_groups]
        df = pd.concat(filtered_groups).reset_index(drop=True)
        
    df_fig = df.copy()
    df_fig["year"] = df_fig["range_impact"].apply(
        lambda x: [year for year in range(int(x[0]), int(x[1]) + 10, 10)]
    )
    df_fig = df_fig.explode("year")

    df_fig = (
        df_fig.groupby(["region_code", "year"])["cultural_score"].sum().reset_index()
    )

    return df_fig

def interpolate_function(df, value = 'value', category = 'region_code', year = 'year'):

    df_interpolated_list = []
    for reg in set(df[category]):

        res = df[df[category]==reg]

        min_date = min(res['year'])
        max_date = max(res['year'])

        year_range = np.arange(min_date, max_date, 1)
        df_year = pd.DataFrame(year_range, columns = [year]).reset_index(drop=True)

        res = pd.merge(res, df_year, on = year, how = 'outer')
        res[category] = reg
        res = res.sort_values(year, ascending=True
                             )
        res[f'{value}_interpolated'] = res[value].interpolate(method='linear')
        res = res.reset_index(drop=True)
        res = res[~res[f'{value}_interpolated'].isna()]

        df_interpolated_list.append(res)
        
    df_interpolated = pd.concat([x for x in df_interpolated_list])
    df_interpolated = df_interpolated.reset_index(drop=True)
    df_interpolated = df_interpolated.drop(value, axis=1)
    

    return df_interpolated


def create_stats_dataset(individuals, type = 'all', top_percent = None):


    df_trend = create_df_trend(individuals, top_percent = top_percent)

    df_maddison = get_maddison_data(data_path_environment)
    df_maddison = df_maddison.drop('country_code_maddison', axis=1)

    df_population = get_population(data_path_environment)

    df_trend = df_trend[df_trend['year']<=1850]

    df_trend_interpolated = interpolate_function(df_trend, value = 'cultural_score', category = 'region_code', year = 'year')
    df_pop_interpolated = interpolate_function(df_population, value = 'population', category = 'region_code', year = 'year')

    final_df = pd.merge(df_trend_interpolated, df_maddison, on = ['region_code', 'year'])
    final_df = pd.merge(final_df, df_pop_interpolated , on =  ['region_code', 'year'])

    final_df['cultural_score_per_capita_interpolated'] = final_df['cultural_score_interpolated']/final_df['population_interpolated']
    final_df['gdp_interpolated'] = final_df['gdp_per_capita']*final_df['population_interpolated']

    min_max_scaler = preprocessing.MinMaxScaler()
    final_df['cultural_score_per_capita_interpolated_norm'] =  min_max_scaler.fit_transform(final_df[['cultural_score_per_capita_interpolated']])
    final_df['cultural_score_interpolared_norm'] =  min_max_scaler.fit_transform(final_df[['cultural_score_interpolated']])
    final_df['gdp_per_capita_norm'] =  min_max_scaler.fit_transform(final_df[['gdp_per_capita']])
    final_df['population_interpolated_norm'] =  min_max_scaler.fit_transform(final_df[['population_interpolated']])
    final_df['gdp_norm'] =  min_max_scaler.fit_transform(final_df[['gdp_interpolated']])

    return final_df

In [261]:
individuals = load_model(
        Individual, name=checkpoint_path + "/individuals.jsonl"
    )


In [ ]:
def create_stats_dataset(type = 'all', top_percent = None):

    individuals = load_model(
        Individual, name=checkpoint_path + "/individuals.jsonl"
    )

    df_trend = create_df_trend(individuals, top_percent = top_percent)

    df_maddison = get_maddison_data(data_path_environment)
    df_maddison = df_maddison.drop('country_code_maddison', axis=1)

    df_population = get_population(data_path_environment)

    df_trend = df_trend[df_trend['year']<=1850]

    df_trend_interpolated = interpolate_function(df_trend, value = 'cultural_score', category = 'region_code', year = 'year')
    df_pop_interpolated = interpolate_function(df_population, value = 'population', category = 'region_code', year = 'year')

    final_df = pd.merge(df_trend_interpolated, df_maddison, on = ['region_code', 'year'])
    final_df = pd.merge(final_df, df_pop_interpolated , on =  ['region_code', 'year'])

    final_df['cultural_score_per_capita_interpolated'] = final_df['cultural_score_interpolated']/final_df['population_interpolated']
    final_df['gdp_interpolated'] = final_df['gdp_per_capita']*final_df['population_interpolated']

    min_max_scaler = preprocessing.MinMaxScaler()
    final_df['cultural_score_per_capita_interpolated_norm'] =  min_max_scaler.fit_transform(final_df[['cultural_score_per_capita_interpolated']])
    final_df['cultural_score_interpolared_norm'] =  min_max_scaler.fit_transform(final_df[['cultural_score_interpolated']])
    final_df['gdp_per_capita_norm'] =  min_max_scaler.fit_transform(final_df[['gdp_per_capita']])
    final_df['population_interpolated_norm'] =  min_max_scaler.fit_transform(final_df[['population_interpolated']])
    final_df['gdp_norm'] =  min_max_scaler.fit_transform(final_df[['gdp_interpolated']])

    return final_df

In [262]:
test = create_df_trend(individuals)

In [265]:
df_stats_top_10 = create_stats_dataset(individuals)

In [266]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.arima.model import ARIMA

model = smf.mixedlm("gdp_per_capita ~ cultural_score_per_capita_interpolated +  year", 
                    df_stats_top_10, 
                    groups=df_stats_top_10["region_code"])
result = model.fit(reml=False)
print(result.aic)
print(result.summary())

54841.53414237001
                         Mixed Linear Model Regression Results
Model:                     MixedLM          Dependent Variable:          gdp_per_capita
No. Observations:          3702             Method:                      ML            
No. Groups:                22               Scale:                       154761.3006   
Min. group size:           1                Log-Likelihood:              -27415.7671   
Max. group size:           600              Converged:                   Yes           
Mean group size:           168.3                                                       
---------------------------------------------------------------------------------------
                                         Coef.    Std.Err.   z    P>|z|  [0.025  0.975]
---------------------------------------------------------------------------------------
Intercept                                 563.351  140.246  4.017 0.000 288.475 838.228
cultural_score_per_capita_interpolated 

In [241]:
model = smf.mixedlm("gdp_per_capita ~ cultural_score_per_capita_interpolated +  year", 
                    df_stats_science, 
                    groups=df_stats_science["region_code"])

result = model.fit(reml=False)
print(result.aic)
print(result.summary())

54620.38444999204
                         Mixed Linear Model Regression Results
Model:                      MixedLM          Dependent Variable:          gdp_per_capita
No. Observations:           3699             Method:                      ML            
No. Groups:                 22               Scale:                       147491.2016   
Min. group size:            1                Log-Likelihood:              -27305.1922   
Max. group size:            600              Converged:                   Yes           
Mean group size:            168.1                                                       
----------------------------------------------------------------------------------------
                                         Coef.    Std.Err.   z    P>|z|  [0.025  0.975] 
----------------------------------------------------------------------------------------
Intercept                                 844.807  143.975  5.868 0.000 562.622 1126.993
cultural_score_per_capita_int

In [242]:
model = smf.mixedlm("gdp_per_capita ~ cultural_score_per_capita_interpolated +  year", 
                    df_stats_90, 
                    groups=df_stats_90["region_code"])
result = model.fit(reml=False)
print(result.aic)
print(result.summary())

54806.03258092314
                         Mixed Linear Model Regression Results
Model:                     MixedLM          Dependent Variable:          gdp_per_capita
No. Observations:          3699             Method:                      ML            
No. Groups:                22               Scale:                       155136.9399   
Min. group size:           1                Log-Likelihood:              -27398.0163   
Max. group size:           600              Converged:                   Yes           
Mean group size:           168.1                                                       
---------------------------------------------------------------------------------------
                                         Coef.    Std.Err.   z    P>|z|  [0.025  0.975]
---------------------------------------------------------------------------------------
Intercept                                 578.956  143.789  4.026 0.000 297.135 860.778
cultural_score_per_capita_interpolated 

In [246]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [248]:
model = SARIMAX(endog=df_stats["gdp_per_capita"], exog=df_stats[["cultural_score_per_capita_interpolated", "year"]], order=(1, 1, 1), exog_re=df_stats["region_code"], mle_regression=False)
result = model.fit()

 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            3     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.48109D+00    |proj g|=  1.41355D-02

At iterate    5    f=  6.48045D+00    |proj g|=  3.08822D-02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    3      6     22      1     0     0   1.783D-02   6.480D+00
  F =   6.4804543397501932     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


In [249]:
print(result.summary())

                               SARIMAX Results                                
Dep. Variable:         gdp_per_capita   No. Observations:                 3702
Model:               SARIMAX(1, 1, 1)   Log Likelihood              -23990.642
Date:                Tue, 18 Apr 2023   AIC                          47987.284
Time:                        15:03:49   BIC                          48005.931
Sample:                             0   HQIC                         47993.920
                               - 3702                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.4490      0.030     14.773      0.000       0.389       0.509
ma.L1         -0.6660      0.027    -24.676      0.000      -0.719      -0.613
sigma2      2.518e+04    122.795    205.019      0.0

In [126]:
import plotly.express as px

In [136]:
df_stats['log_capita'] = np.log(1 + df_stats['cultural_score_per_capita_interpolated_norm'])
df_stats['log_gdp'] = np.log(1 + df_stats['gdp_per_capita_norm'])


/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_93899/2306580304.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/lx/9b13r6ln63jb54_3xlf9b_380000gn/T/ipykernel_93899/2306580304.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [137]:
fig = px.scatter(df_stats, x = 'log_capita', y = 'log_gdp', color = 'region_code', width = 2000, height = 1000, trendline="ols")

In [138]:
import plotly

In [139]:
plotly.offline.plot(fig)

'temp-plot.html'

In [119]:
df_stats[df_stats['region_code']=='re_latin']

,region_code,year,cultural_score_interpolated,gdp_per_capita,population_interpolated,cultural_score_per_capita_interpolated,gdp_interpolated,cultural_score_per_capita_interpolated_norm,cultural_score_interpolared_norm,gdp_per_capita_norm,population_interpolated_norm,gdp_norm
597,re_latin,1,0.217161,1407,25.206,0.008615,35464.842,0.000487,0.000318,0.178982,0.056861,0.093373
598,re_latin,1310,0.144284,2857,21.450,0.006727,61282.650,0.000368,0.000120,0.499779,0.048216,0.162673
599,re_latin,1311,0.144284,2750,21.450,0.006727,58987.500,0.000368,0.000120,0.476106,0.048216,0.156512
600,re_latin,1312,0.144284,2788,21.450,0.006727,59802.600,0.000368,0.000120,0.484513,0.048216,0.158700
601,re_latin,1313,0.144284,2788,21.450,0.006727,59802.600,0.000368,0.000120,0.484513,0.048216,0.158700
...,...,...,...,...,...,...,...,...,...,...,...,...
1133,re_latin,1845,0.144284,2673,21.450,0.006727,57335.850,0.000368,0.000120,0.459071,0.048216,0.152079
1134,re_latin,1846,0.144284,2650,21.450,0.006727,56842.500,0.000368,0.000120,0.453982,0.048216,0.150754
1135,re_latin,1847,0.144284,2588,21.450,0.006727,55512.600,0.000368,0.000120,0.440265,0.048216,0.147185
1136,re_latin,1848,0.144284,2657,21.450,0.006727,56992.650,0.000368,0.000120,0.455531,0.048216,0.151157


In [121]:
df_maddison[df_maddison['region_code']=='re_latin']

,region_code,year,gdp_per_capita
0,re_latin,1,1407
1,re_latin,1310,2857
2,re_latin,1311,2750
3,re_latin,1312,2788
4,re_latin,1313,2788
...,...,...,...
705,re_latin,2014,32829
706,re_latin,2015,33118
707,re_latin,2016,33419
708,re_latin,2017,34027
